In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import zipfile

train_df_promt = pd.read_csv('prompts_train.csv')
train_df = pd.read_csv('summaries_train.csv')
test_df_promt = pd.read_csv('prompts_test.csv')
test_df = pd.read_csv('summaries_test.csv')
print(f'Size train set: {train_df.shape} \nSize test set: {test_df.shape}')
print(f'Size train promts: {train_df_promt.shape} \nSize test promts: {test_df_promt.shape}')

import re
import string

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

!pip install transformers torch

from transformers import BertTokenizer, BertModel, BertConfig, AdamW, get_linear_schedule_with_warmup, BertForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from sklearn.metrics import mean_squared_error
import torch.nn as nn

Size train set: (7165, 5) 
Size test set: (4, 3)
Size train promts: (4, 4) 
Size test promts: (2, 4)


In [ ]:
# def truncate_text(text, max_length=40):
#     """Truncates the string to the specified number of characters"""
#     if len(text) > max_length:
#         return text[:max_length] + '...'
#     return text

train_sample_promt = train_df_promt.head(4).copy()
test_sample_promt = test_df_promt.head(2).copy()
train_sample = train_df.head(4).copy()
test_sample = test_df.head(2).copy()

# train_sample_promt.loc[:, 'prompt_text'] = train_sample_promt['prompt_text'].apply(truncate_text)
# train_sample_promt.loc[:, 'prompt_question'] = train_sample_promt['prompt_question'].apply(truncate_text)
# test_sample_promt.loc[:, 'prompt_text'] = test_sample_promt['prompt_text'].apply(truncate_text)
# train_sample.loc[:, 'text'] = train_sample['text'].apply(truncate_text)

In [ ]:
df_train = train_df.merge(train_df_promt, on='prompt_id')

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def clean_text(text):
    # Приведение к нижнему регистру
    text = text.lower()
    # Удаление знаков препинания, специальных символов и чисел
    text = re.sub(r'[^a-z\s]', '', text)
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# def tokenize_text(text):
#     tokens = word_tokenize(text)
#     return tokens

stop_words = {
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
    "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she",
    "her", "hers", "herself", "it", "its", "itself", "they", "them", "their",
    "theirs", "themselves", "what", "which", "who", "whom", "this", "that",
    "these", "those", "am", "is", "are", "was", "were", "be", "been", "being",
    "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an",
    "the", "and", "but", "if", "or", "because", "as", "until", "while", "of",
    "at", "by", "for", "with", "about", "against", "between", "into", "through",
    "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then",
    "once", "here", "there", "when", "where", "why", "how", "all", "any",
    "both", "each", "few", "more", "most", "other", "some", "such", "no",
    "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s",
    "t", "can", "will", "just", "don", "should", "now"
}

def remove_stopwords(tokens):
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

# from nltk.stem import WordNetLemmatizer

# def lemmatize_text(tokens):
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
#     return lemmatized_tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
df_train['text'] = df_train['text'].apply(clean_text)
# df_train['text'] = df_train['text'].apply(tokenize_text)
df_train['text'] = df_train['text'].apply(remove_stopwords)
# df_train['text'] = df_train['text'].apply(lemmatize_text)

In [ ]:
X = df_train['text']  # Признаки: текст
y_content = df_train['content']  # Целевая переменная 1: content
y_wording = df_train['wording']  # Целевая переменная 2: wording

X_train, X_test, y_train_content, y_test_content, y_train_wording, y_test_wording = train_test_split(
    X, y_content, y_wording, test_size=0.2, random_state=42)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_texts(texts, tokenizer, max_len=40):
    inputs = tokenizer.batch_encode_plus(
        texts,
        max_length=max_len,
        padding=True,
        truncation=True,
        return_tensors='pt'
    )
    return inputs

train_inputs = encode_texts(X_train.tolist(), tokenizer)
test_inputs = encode_texts(X_test.tolist(), tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
y_train_combined = torch.stack((torch.tensor(y_train_content.values, dtype=torch.float32),
                                torch.tensor(y_train_wording.values, dtype=torch.float32)), dim=1)

y_test_combined = torch.stack((torch.tensor(y_test_content.values, dtype=torch.float32),
                               torch.tensor(y_test_wording.values, dtype=torch.float32)), dim=1)

# Создание набора данных и загрузчика данных
class SummaryDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SummaryDataset(train_inputs, y_train_combined)
test_dataset = SummaryDataset(test_inputs, y_test_combined)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)


# Загрузка модели BERT с дополнительным линейным слоем для регрессии

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Оптимизатор и график обучения

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
epochs = 3
total_steps = len(train_dataloader) * 3  # 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

# Обучение модели


In [ ]:
for epoch in range(epochs):
    print(f'Epoch {epoch + 1}/{epochs}')

    # Training
    model.train()
    total_loss = 0

    for batch in train_dataloader:
        b_input_ids = batch['input_ids'].to(device)
        b_attention_mask = batch['attention_mask'].to(device)
        b_labels = batch['labels'].to(device)

        model.zero_grad()

        outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f'Average training loss: {avg_train_loss:.2f}')

    # Validation
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in test_dataloader:
            b_input_ids = batch['input_ids'].to(device)
            b_attention_mask = batch['attention_mask'].to(device)
            b_labels = batch['labels'].to(device)

            outputs = model(b_input_ids, attention_mask=b_attention_mask)
            logits = outputs.logits

            predictions.extend(logits.detach().cpu().numpy())
            true_labels.extend(b_labels.cpu().numpy())

# Постобработка и расчет MCRMSE
predictions = np.array(predictions)
true_labels = np.array(true_labels)

def calculate_rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

mcrmse_score_content = calculate_rmse(true_labels[:, 0], predictions[:, 0])
mcrmse_score_wording = calculate_rmse(true_labels[:, 1], predictions[:, 1])

print(f"MCRMSE = {np.mean([mcrmse_score_content, mcrmse_score_wording])}")

Epoch 1/3


<ipython-input-8-ea6290e6bf41>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Average training loss: -2.55
Epoch 2/3
Average training loss: -6.22
Epoch 3/3
Average training loss: -8.83
MCRMSE = 19.044246673583984
